Set Shots - Expected Goal Model - Model Tuning and Building - GBM

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import goal_set_shots_file_path
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Model Version

In [2]:
model_version = 8
model_name = 'expected_goal_set'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [3]:
RESPONSE = ModellingDataContract.RESPONSE_GOAL

In [5]:
FEATURES = ModellingDataContract.feature_list_set_goal

In [6]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints_set_goal

Load Data

In [7]:
df_modelling = pd.read_csv(goal_set_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,Goal,Behind,Miss,Score,Event_Type1,Set_Shot,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,GoalTrainingSet,GoalTestSet,GoalValidationSet
16981,186,possGain,behind,1447.0,4,185,224.0,Collingwood,Collingwood,Bobby Hill,Bobby_Hill,Kick,32.0,0.0,ineffective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,0,1,0,1,Uncontested Mark,True,32.0,48.000000,0.000000,0.133929,False,False,False,True,False,True,False,False
16982,196,possGain,behind,1527.0,4,425,476.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,50.0,-19.0,ineffective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,0,1,0,1,Mark On Lead,True,50.0,35.510562,0.564569,0.153507,False,False,False,True,False,False,False,True
16983,210,throwIn,behind,1610.0,4,804,841.0,Brisbane Lions,Brisbane Lions,Eric Hipwood,Eric_Hipwood,Kick,36.0,37.0,ineffective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,0,1,0,1,Uncontested Mark,True,36.0,57.489129,0.699193,0.085469,False,False,False,False,True,True,False,False
16984,224,possGain,goal,1740.0,4,1290,1348.0,Collingwood,Collingwood,Steele Sidebottom,Steele_Sidebottom,Kick,37.0,34.0,effective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,1,0,0,6,Mark On Lead,True,37.0,54.817880,0.669043,0.091894,False,False,False,True,False,True,False,False
16985,233,ballUp,goal,1811.0,4,1591,1629.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,54.0,2.0,effective,True,NaN,141,160,Collingwood,Brisbane Lions,right,2023F4_Collingwood_BrisbaneLions,2023F4,NaN,2023.0,1,0,0,6,Uncontested Mark,True,54.0,26.076810,0.076772,0.248447,True,False,False,False,False,False,False,True


In [8]:
df_modelling[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(df_modelling['Initial_State'])

In [9]:
training_data = df_modelling[df_modelling[RESPONSE+"TrainingSet"]]
test_data = df_modelling[df_modelling[RESPONSE+"TestSet"]]
cal_data = df_modelling[df_modelling[RESPONSE+"ValidationSet"]]

In [10]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [11]:
y_train.mean(), y_test.mean(), y_cal.mean()

(0.5385464581416743, 0.53708063566804, 0.5647534952170714)

In [12]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season,Behind,Miss,Score,Event_Type1,Set_Shot,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,GoalTrainingSet,GoalTestSet,GoalValidationSet
1,28,possGain,behind,295.0,1,859,911.0,Sydney,Sydney,Sam Reid,Sam_Reid,Kick,52.0,35.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,1,0,1,Contested Mark,True,52.0,43.600459,0.931882,0.088007,False,False,False,True,False,False,True,False
17,168,centreBounce,miss,1480.0,3,1850,1899.0,Brisbane Lions,Brisbane Lions,Eric Hipwood,Eric_Hipwood,Kick,52.0,-31.0,clanger,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,0,1,0,Mark On Lead,True,52.0,40.459857,0.872894,0.102289,False,True,False,False,False,False,True,False
19,189,kickIn,goal,1639.0,4,390,439.0,Brisbane Lions,Brisbane Lions,Lincoln McCarthy,Lincoln_McCarthy,Kick,55.0,-2.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN,0,0,6,Free For,True,55.0,23.086793,0.086738,0.281582,False,False,True,False,False,False,True,False
25,11,centreBounce,goal,103.0,1,190,249.0,Western Bulldogs,Western Bulldogs,Aaron Naughton,Aaron_Naughton,Kick,54.0,-18.0,effective,True,NaN,141,160,Collingwood,Western Bulldogs,right,202101_Collingwood_WesternBulldogs,202101,2021.0,NaN,0,0,6,Mark On Lead,True,54.0,31.622777,0.605545,0.168122,False,True,False,False,False,False,True,False
36,139,possGain,behind,1190.0,3,205,256.0,Collingwood,Collingwood,Brody Mihocek,Brody_Mihocek,Kick,50.0,-30.0,ineffective,True,NaN,141,160,Collingwood,Western Bulldogs,right,202101_Collingwood_WesternBulldogs,202101,2021.0,NaN,1,0,1,Uncontested Mark,True,50.0,42.426407,0.785398,0.107277,False,False,False,True,False,False,True,False


Preprocess Data

In [13]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [14]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]
X_cal_preproc = X_cal[FEATURES]

In [15]:
X_train_preproc.shape, X_test_preproc.shape

((10870, 9), (3398, 9))

In [16]:
X_train_preproc.head()

,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn
0,46.0,52.009614,0.908067,0.075999,False,False,True,False,False
2,45.0,36.674242,0.451453,0.158231,False,False,False,True,False
4,33.0,46.572524,0.260602,0.133410,False,True,False,False,False
5,52.0,44.407207,0.945311,0.084822,False,False,False,True,False
6,47.0,46.754679,0.845930,0.091187,False,False,False,True,False


In [17]:
X_test_preproc.head()

,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn
1,52.0,43.600459,0.931882,0.088007,False,False,False,True,False
17,52.0,40.459857,0.872894,0.102289,False,True,False,False,False
19,55.0,23.086793,0.086738,0.281582,False,False,True,False,False
25,54.0,31.622777,0.605545,0.168122,False,True,False,False,False
36,50.0,42.426407,0.785398,0.107277,False,False,False,True,False


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [18]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [19]:
xgb_tuner.training_data.head()

,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn
0,46.0,52.009614,0.908067,0.075999,False,False,True,False,False
2,45.0,36.674242,0.451453,0.158231,False,False,False,True,False
4,33.0,46.572524,0.260602,0.133410,False,True,False,False,False
5,52.0,44.407207,0.945311,0.084822,False,False,False,True,False
6,47.0,46.754679,0.845930,0.091187,False,False,False,True,False


In [20]:
xgb_tuner.tune_hyperparameters()

[I 2023-10-21 13:38:26,204] A new study created in memory with name: no-name-fd4ab348-04c8-489f-b8f1-97c7c5975b2c
[I 2023-10-21 13:38:26,252] Trial 0 finished with value: 0.6023059962422815 and parameters: {'max_depth': 13, 'min_child_weight': 78, 'eta': 0.9784216637337302, 'gamma': 0.00115693452356851, 'lambda': 0.00032230132809928487, 'alpha': 0.0034831918621882904, 'subsample': 0.8229577102356391, 'colsample_bytree': 0.5134737741254563}. Best is trial 0 with value: 0.6023059962422815.
[I 2023-10-21 13:38:26,282] Trial 1 finished with value: 0.605725005377101 and parameters: {'max_depth': 16, 'min_child_weight': 13, 'eta': 0.36020116726469037, 'gamma': 0.05953876363033122, 'lambda': 0.0017161362432020185, 'alpha': 0.009804746099725542, 'subsample': 0.2054960280816995, 'colsample_bytree': 0.540691453086424}. Best is trial 0 with value: 0.6023059962422815.
[I 2023-10-21 13:38:26,308] Trial 2 finished with value: 0.6012842256411669 and parameters: {'max_depth': 20, 'min_child_weight': 8

Number of finished trials:  1000
Best trial:
  Value: 0.5755499976965067
  Params: 
    max_depth: 18
    min_child_weight: 10
    eta: 0.6450950877705423
    gamma: 0.015562976956132275
    lambda: 0.00018379407959063954
    alpha: 0.577545041504984
    subsample: 0.783377959055626
    colsample_bytree: 0.3996090457164005


In [21]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 18,
 'min_child_weight': 10,
 'eta': 0.6450950877705423,
 'gamma': 0.015562976956132275,
 'lambda': 0.00018379407959063954,
 'alpha': 0.577545041504984,
 'subsample': 0.783377959055626,
 'colsample_bytree': 0.3996090457164005}

Training Model - SuperXGBClassifier class for training and predictions

In [22]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [23]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_cal_preproc,
                               y_cal = y_cal,
                               params = params)

In [24]:
super_xgb.fit()

[0]	validation_0-logloss:0.66528	validation_1-logloss:0.66631
[1]	validation_0-logloss:0.60991	validation_1-logloss:0.61416
[2]	validation_0-logloss:0.59788	validation_1-logloss:0.60298
[3]	validation_0-logloss:0.59466	validation_1-logloss:0.59957
[4]	validation_0-logloss:0.59220	validation_1-logloss:0.59754
[5]	validation_0-logloss:0.59190	validation_1-logloss:0.59732
[6]	validation_0-logloss:0.59163	validation_1-logloss:0.59763
[7]	validation_0-logloss:0.59142	validation_1-logloss:0.59762
[8]	validation_0-logloss:0.59129	validation_1-logloss:0.59766
[9]	validation_0-logloss:0.59078	validation_1-logloss:0.59782
[10]	validation_0-logloss:0.59067	validation_1-logloss:0.59793
[11]	validation_0-logloss:0.59059	validation_1-logloss:0.59762
[12]	validation_0-logloss:0.59053	validation_1-logloss:0.59769
[13]	validation_0-logloss:0.59052	validation_1-logloss:0.59818
[14]	validation_0-logloss:0.59052	validation_1-logloss:0.59842
[15]	validation_0-logloss:0.59056	validation_1-logloss:0.59859
[1

In [25]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.3996090457164005, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.015562976956132275, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.6450950877705423, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=18, max_leaves=None,
              min_child_weight=10, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': -1,
                                    'Distance_to_Middle_Goal': -1,
                                    'Visible_Goal_Angle': 1, 'x0': -1},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [26]:
super_xgb.xgb_model.get_booster().feature_names

['x0',
 'Distance_to_Middle_Goal',
 'Angle_to_Middle_Goal',
 'Visible_Goal_Angle',
 'ballUp',
 'centreBounce',
 'kickIn',
 'possGain',
 'throwIn']

In [27]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [28]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
cal_probas = super_xgb.predict_proba(X_cal_preproc)[:, 1]

In [29]:
super_xgb.calibrate()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [30]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)
cal_cal_probas = super_xgb.predict_proba(X_cal_preproc, calibrate=True)

Check Average Predictions

In [31]:
train_probas.mean(), training_data[RESPONSE].mean(), train_cal_probas.mean()

(0.53925985, 0.5385464581416743, 0.5572183211786965)

In [32]:
test_probas.mean(), test_data[RESPONSE].mean(), test_cal_probas.mean()

(0.5372323, 0.53708063566804, 0.5553080744347371)

In [33]:
cal_probas.mean(), cal_data[RESPONSE].mean(), cal_cal_probas.mean()

(0.5477017, 0.5647534952170714, 0.5647543332824958)

Export model

In [34]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [35]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export['xgb_probas_cal'] = train_cal_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Goal,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,17,183.0,0,46.0,52.009614,0.908067,0.075999,False,False,True,False,False,0,0.222003,0.284641
1,202101_BrisbaneLions_Sydney,34,354.0,0,45.0,36.674242,0.451453,0.158231,False,False,False,True,False,1,0.659142,0.655705
2,202101_BrisbaneLions_Sydney,59,578.0,1,33.0,46.572524,0.260602,0.133410,False,True,False,False,False,1,0.552690,0.558518
3,202101_BrisbaneLions_Sydney,62,621.0,1,52.0,44.407207,0.945311,0.084822,False,False,False,True,False,0,0.297784,0.346515
4,202101_BrisbaneLions_Sydney,72,696.0,1,47.0,46.754679,0.845930,0.091187,False,False,False,True,False,0,0.360239,0.396961


In [36]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export['xgb_probas_cal'] = test_cal_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Goal,x0,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,ballUp,centreBounce,kickIn,possGain,throwIn,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,28,295.0,0,52.0,43.600459,0.931882,0.088007,False,False,False,True,False,0,0.307412,0.354290
1,202101_BrisbaneLions_Sydney,168,1480.0,0,52.0,40.459857,0.872894,0.102289,False,True,False,False,False,0,0.441856,0.463809
2,202101_BrisbaneLions_Sydney,189,1639.0,1,55.0,23.086793,0.086738,0.281582,False,False,True,False,False,1,0.854518,0.852552
3,202101_Collingwood_WesternBulldogs,11,103.0,1,54.0,31.622777,0.605545,0.168122,False,True,False,False,False,1,0.669661,0.665694
4,202101_Collingwood_WesternBulldogs,139,1190.0,0,50.0,42.426407,0.785398,0.107277,False,False,False,True,False,0,0.387665,0.419242
